In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import HTML

import numpy as np
import numpy.linalg as la
from scipy.integrate import RK45

import sympy as sym
# Heaviside numerics workaround
sympy_modules = [{'Heaviside': lambda x: np.heaviside(x,0.5)}, 'numpy']

from table_maker import *
from functools import partial, reduce
from itertools import *
from math import ceil

import pickle

def cos_bell(x, center=0, width=2*np.pi, height=1):
    return (np.cos((x-center)/width*2*np.pi)+1)/2*height * np.heaviside(x-center+width/2,0) * np.heaviside(-x+center+width/2,0)

import collections.abc
def collect_eqn(expr, terms):
    if not isinstance(terms, collections.abc.Container):
        term = terms # isn't a list or tuple
        left, right = 0,0
        for my_expr in expr.args:
            has_term = False
            for my_term in sym.postorder_traversal(my_expr):
                if term == my_term:
                    has_term = True
                    break
            if has_term:
                left += my_expr
            else:
                right -= my_expr
        return sym.Eq(left, right, evaluate=False)
    else: # isn't a list or tuple
        left_total, right_total = 0, expr
        for term in terms:
            left, right = collect_eqn(right_total, term).args
            left_total += left
            right_total = -right
        return sym.Eq(left_total, right_total, evaluate=False)
    
def my_print(expr):
    global eviron_print_tex
    if eviron_print_tex:
        print('$$' + sym.latex(expr) + '$$')
    else:
        display(expr)
        
def batch_print(expr_list, punctuation='.', return_string=False):
    global eviron_print_tex
    if eviron_print_tex:
        print_str = '$$\\begin{align*}\n'
        for expr in expr_list:
            print_str += '\t' + sym.latex(expr).replace('=','&=') + '\\\\\n'
        # remove last new-line and add ending puncuation
        print_str = print_str[:-3] + punctuation + '\n'
        print_str += '\\end{align*}$$'
        if return_string:
            return print_str
        else:
            print(print_str)
    else:
        for expr in expr_list:
            display(expr)
            
def py_code(expr, replace_numpy=True, replace_unicode=True, numpy_package='np.'):
    code = sym.ccode(expr)
    
    if replace_unicode:
        unicode_replacements = [
            ('\\alpha', 'α'),
            ('\\beta', 'β'),
            ('\\gamma', 'γ'),
            ('\\Gamma', 'Γ'),
            ('\\delta', 'δ'),
            ('\\Delta', 'Δ'),
            ('\\epsilon', 'ϵ'),
            ('\\zeta', 'ζ'),
            ('\\eta', 'η'),
            ('\\theta', 'θ'),
            ('\\xi', 'ξ'),
            ('\\mu', 'μ')
        ]
        for unicode, symbol in unicode_replacements:
            code = code.replace(unicode, symbol)
    
    if replace_numpy:
        numpy_replacements = [
            ('exp', numpy_package + 'exp'),
            ('log', numpy_package + 'log'),
            ('pow', '( lambda base, exponent: base**exponent )'),
            ('Heaviside', '(lambda input: np.heaviside(input,0.5))')
        ]
        for c_func, np_func in numpy_replacements:
            code = code.replace(c_func, np_func)
        
    return code

In [2]:
import matplotlib
matplotlib.rcParams.update({'font.size': 24})

In [155]:
eviron_print_tex = True

μ, α, γ, θ, c, Δ, λ = sym.symbols('\\mu \\alpha \\gamma, \\theta, c \\Delta \\lambda', real=True, positive = True)

ξ = sym.symbols('\\xi', real=True)

U = sym.Function('U', real=True)(ξ)
A = sym.Function('A', real=True)(ξ)

LU = lambda f: c*μ*f.diff(ξ) - f
LA = lambda f: c*α*f.diff(ξ) - f

coeff_left_pos = (sym.exp(Δ) - 1)/2 * sym.exp(-λ)/(1-sym.exp(-λ)) + sym.Rational(1,2)
coeff_left_neg = (1 - sym.exp(-Δ))/2 * sym.exp(-λ)/(1-sym.exp(-λ)) - sym.exp(-Δ)/2
F_left = coeff_left_pos * sym.exp(ξ) + coeff_left_neg*sym.exp(-ξ)

coeff_right_pos = (sym.exp(Δ)-1)/2*sym.exp(-λ)/(1-sym.exp(-λ))
coeff_right_neg = (1-sym.exp(-Δ))/2/(1-sym.exp(-λ))
F_right = coeff_right_pos * sym.exp(ξ) + coeff_right_neg*sym.exp(-ξ)

G_left = γ
G_right = 0

# homogeneous
Uhom = sym.dsolve(LU(U), U).args[1]
Ahom = sym.dsolve(LA(A), A).args[1]
B, C = sym.symbols('B C', real=True)
C1, C2, C3, C4 = sym.symbols('C1 C2 C3 C4')

# U left
ansatz = B*sym.exp(ξ) + C*sym.exp(-ξ)
L_ansatz = LU(ansatz).expand().collect(sym.exp(ξ))
B_sub = coeff_left_pos/-L_ansatz.collect(B).coeff(B*sym.exp(ξ))
C_sub = coeff_left_neg/-L_ansatz.collect(C).coeff(C*sym.exp(-ξ))
U_particular_left = B_sub*sym.exp(ξ) + C_sub*sym.exp(-ξ)
assert (LU(U_particular_left) + F_left).simplify() == 0
U_left = Uhom.expand().subs(sym.exp(C1/μ/c), C1) + U_particular_left

# U right
ansatz = B*sym.exp(ξ) + C*sym.exp(-ξ)
L_ansatz = LU(ansatz).expand().collect(sym.exp(ξ))
B_sub = coeff_right_pos/-L_ansatz.collect(B).coeff(B*sym.exp(ξ))
C_sub = coeff_right_neg/-L_ansatz.collect(C).coeff(C*sym.exp(-ξ))
U_particular_right = B_sub*sym.exp(ξ) + C_sub*sym.exp(-ξ)
assert (LU(U_particular_right) + F_right).simplify() == 0
U_right = Uhom.expand().subs(sym.exp(C1/μ/c), C2) + U_particular_right

# A left
L_ansatz = LA(B)
B_sub = G_left/-L_ansatz.coeff(B)
A_particular_left = B_sub
assert (LA(A_particular_left) + G_left).simplify() == 0
A_left = Ahom.expand().subs(sym.exp(C1/α/c), C3) + A_particular_left

# A right
A_right = Ahom.expand().subs(sym.exp(C1/α/c), C4)

# piecewise - for printing
U_piecewise = sym.Piecewise( (U_left, ξ<0), (U_right, 0<= ξ) )
A_piecewise = sym.Piecewise( (A_left, ξ<0), (A_right, 0<= ξ) )

print('Solving the system, we obtain the solution in terms of 7 unkown constants')
batch_print([ sym.Eq(U, U_piecewise), sym.Eq(A, A_piecewise) ])

expr1 = U_left.subs(ξ,0) - U_right.subs(ξ,0)
row1 = [expr1.coeff(C1), expr1.coeff(C2)]
rhs1 = row1[0]*C1 + row1[1]*C2 - expr1

expr2 = U_left.subs(ξ,-Δ) - U_right.subs(ξ,λ-Δ)
row2 = [expr2.coeff(C1), expr2.coeff(C2)]
rhs2 = row2[0]*C1 + row2[1]*C2 - expr2

mat = sym.Matrix([row1, row2])
rhs = sym.Matrix([[rhs1], [rhs2]])

print('Enforcing continuity of $U$ at $\\xi=0$ and over the boundary $\\xi = -\\Delta, \\lambda-\\Delta$ we obtain the following equations for $C_1$ and $C_2$')
print('$$')
print(sym.latex(mat) + sym.latex(sym.Matrix([[C1],[C2]])) + '=' + sym.latex(rhs))
print('$$')

C1_sub, C2_sub = mat**-1 * rhs

print('which gives')
batch_print([sym.Eq(C1, C1_sub), sym.Eq(C2, C2_sub)])

expr1 = A_left.subs(ξ,0) - A_right.subs(ξ,0)
row1 = [expr1.coeff(C3), expr1.coeff(C4)]
rhs1 = row1[0]*C3 + row1[1]*C4 - expr1

expr2 = A_left.subs(ξ,-Δ) - A_right.subs(ξ,λ-Δ)
row2 = [expr2.coeff(C3), expr2.coeff(C4)]
rhs2 = row2[0]*C3 + row2[1]*C4 - expr2

mat = sym.Matrix([row1, row2])
rhs = sym.Matrix([[rhs1], [rhs2]])

print('Enforcing continuity of $A$ at $\\xi=0$ and over the boundary $\\xi = -\\Delta, \\lambda-\\Delta$ we obtain the following equations for $C_3$ and $C_4$')
print('$$')
print(sym.latex(mat) + sym.latex(sym.Matrix([[C3],[C4]])) + '=' + sym.latex(rhs))
print('$$')

C3_sub, C4_sub = mat**-1 * rhs

print('which gives')
batch_print([sym.Eq(C3, C3_sub), sym.Eq(C4, C4_sub)])

print('We now have the solution in terms of $\\lambda, \\Delta$ and $c$. Enforcing threshold crossing of $U-A$ at $\\xi = 0, -\\Delta$ gives')
cond1 = (U_right - A_right - θ).subs({ξ:0, C2:C2_sub, C4:C4_sub})
cond2 = (U_right - A_right - θ).subs({ξ:-Δ, C2:C2_sub, C4:C4_sub})
batch_print([sym.Eq(0,cond1), sym.Eq(0,cond2)])

Solving the system, we obtain the solution in terms of 7 unkown constants
$$\begin{align*}
	U{\left(\xi \right)} &= \begin{cases} C_{1} e^{\frac{\xi}{\mu c}} + \frac{\left(\frac{1}{2} + \frac{\left(\frac{e^{\Delta}}{2} - \frac{1}{2}\right) e^{- \lambda}}{1 - e^{- \lambda}}\right) e^{\xi}}{- \mu c + 1} + \frac{\left(\frac{\left(\frac{1}{2} - \frac{e^{- \Delta}}{2}\right) e^{- \lambda}}{1 - e^{- \lambda}} - \frac{e^{- \Delta}}{2}\right) e^{- \xi}}{\mu c + 1} & \text{for}\: \xi < 0 \\C_{2} e^{\frac{\xi}{\mu c}} + \frac{\left(\frac{1}{2} - \frac{e^{- \Delta}}{2}\right) e^{- \xi}}{\left(1 - e^{- \lambda}\right) \left(\mu c + 1\right)} + \frac{\left(\frac{e^{\Delta}}{2} - \frac{1}{2}\right) e^{- \lambda} e^{\xi}}{\left(1 - e^{- \lambda}\right) \left(- \mu c + 1\right)} & \text{otherwise} \end{cases}\\
	A{\left(\xi \right)} &= \begin{cases} C_{3} e^{\frac{\xi}{\alpha c}} + \gamma & \text{for}\: \xi < 0 \\C_{4} e^{\frac{\xi}{\alpha c}} & \text{otherwise} \end{cases}.
\end{align*}$$
Enforcing c